# User-based CF

This is a step-by-step demonstration (and hands-on activity) of building a user-based collaborative filtering from scratch, i.e. without using any libraries. These steps simulate the process of generating recommendations for one particular user (ref_user), given a number of neighbors (top-n similar users), a similarity metric, and a prediction formula. 

Note that this is not the optimal way of dealing with this problem (also, there are more than one ways to implement the following steps). Instead, we want to be able to generate recommendations by predicting missing ratings for the entire utility matrix, not for each user one by one. 

### Import libraries

In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd

### Import Dataset

In [2]:
df = pd.read_csv('/Users/ankitadeshmukh/Desktop/SJSU/Academic/Fall22/CMPE256/Assignments/Movienight-Fall22-ratings.csv')
columns = list(df.keys())
columns.remove('ID')
df.head()

,ID,Rating [Rogue One/Star Wars],Rating [Fight Club],Rating [The Lord of the Rings],Rating [Trolls],Rating [Despicable Me],Rating [Soul],Rating [The hangover],Rating [Captain America],Rating [The big Lebowski],...,Rating [The hunger games],Rating [Pulp Fiction],Rating [Sing ],Rating [500 days of Summer],Rating [Twilight],Rating [Lala Land],Rating [A Fantastic Woman],Rating [Loveless],Rating [The Insult],Rating [The Square]
0,492,4.0,5.0,3.0,NaN,2.0,NaN,NaN,5.0,NaN,...,3.0,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
1,437,3.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0
2,845,5.0,5.0,3.0,NaN,5.0,NaN,5.0,3.0,NaN,...,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,179,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0
4,275,NaN,NaN,4.0,NaN,5.0,NaN,3.0,5.0,NaN,...,4.0,NaN,3.0,5.0,2.0,4.0,NaN,NaN,NaN,NaN


In [3]:
user_dict = {}
for i in range(len(df)):
    user_dict[int(df.iloc[i].ID)] = int(df.iloc[i].ID)

In [4]:
# movie_df = pd.read_csv("movie_name.csv")
movie_df = pd.read_csv('/Users/ankitadeshmukh/Desktop/SJSU/Academic/Fall22/CMPE256/Assignments/movie_name.csv')

movie_dict = {}
for i in range(len(movie_df)):
    movie_dict[movie_df.iloc[i].id] = movie_df.iloc[i].movieName

### Define functions for similarity

In [5]:
#simple cosine
def cosine(x, y):
    sim = np.dot(x,y)/(norm(x)*norm(y))
    return sim

In [6]:
#simple pearson
def pearson(x,y):
    sim = np.dot(x-np.mean(x),y-np.mean(y))/(norm(x-np.mean(x))*norm(y-np.mean(y)))
    return sim

### Convert data to vectors
Deal with NaN values as well -- turn them into 0s

In [7]:
#An array of arrays. Each row (array) represents a user vector. This is in essence the utility matrix. 

u_vector = df.iloc[:,1:].to_numpy()
u_vector[np.isnan(u_vector)] = 0
u_vector

array([[4., 5., 3., 0., 2., 0., 0., 5., 0., 0., 3., 4., 0., 0., 5., 0.,
        0., 0., 0., 0.],
       [3., 4., 5., 5., 5., 5., 5., 4., 4., 4., 5., 5., 5., 5., 5., 5.,
        5., 4., 4., 4.],
       [5., 5., 3., 0., 5., 0., 5., 3., 0., 0., 5., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 3., 0.,
        0., 0., 0., 3.],
       [0., 0., 4., 0., 5., 0., 3., 5., 0., 0., 4., 0., 3., 5., 2., 4.,
        0., 0., 0., 0.],
       [3., 5., 4., 1., 1., 5., 2., 4., 3., 0., 4., 5., 0., 0., 4., 0.,
        0., 0., 0., 0.],
       [0., 0., 4., 0., 3., 0., 0., 3., 0., 0., 0., 0., 3., 0., 3., 3.,
        0., 0., 0., 0.],
       [0., 0., 4., 0., 4., 0., 5., 4., 0., 0., 0., 0., 0., 0., 5., 2.,
        0., 0., 0., 0.],
       [0., 5., 5., 0., 5., 0., 5., 5., 0., 0., 3., 5., 0., 0., 3., 5.,
        0., 0., 0., 0.],
       [0., 4., 0., 3., 5., 0., 5., 4., 0., 0., 4., 5., 0., 5., 0., 4.,
        0., 0., 0., 0.],
       [0., 5., 5., 0., 4., 0.

### Find Similarity Matrix

In [8]:
rows = len(u_vector)

#Initialize 2d list
cosine_matrix = [[0]*rows for k in range(rows)]
pearson_matrix = [[0]*rows for k in range(rows)]

for i in range(rows):
    for j in range(rows):
        cosine_matrix[i][j] = cosine(u_vector[i], u_vector[j])
        pearson_matrix[i][j] = pearson(u_vector[i], u_vector[j])

#### Cosine Similarity

In [9]:
cosine_matrix = np.array(cosine_matrix)
cosine_matrix

array([[1.        , 0.58787456, 0.68968654, 0.61048286, 0.50451053,
        0.84823552, 0.5411049 , 0.5666546 , 0.75417729, 0.54890343,
        0.77219955, 0.61838543, 0.5704453 , 0.80716488, 0.18677184,
        0.51913402, 0.30193176, 0.62185981, 0.58565718, 0.49562893],
       [0.58787456, 1.        , 0.57672138, 0.36496485, 0.68805614,
        0.71384966, 0.57409249, 0.55977933, 0.68409263, 0.69291108,
        0.68845617, 0.65395554, 0.54733082, 0.74120805, 0.34462276,
        0.60306097, 0.64863582, 0.69602162, 0.56417177, 0.77464509],
       [0.68968654, 0.57672138, 1.        , 0.39291264, 0.6228411 ,
        0.64618987, 0.41611986, 0.56933484, 0.71981575, 0.64624303,
        0.737343  , 0.51540671, 0.64790132, 0.66329112, 0.47140452,
        0.65052293, 0.19732001, 0.85376558, 0.75588487, 0.52981294],
       [0.61048286, 0.36496485, 0.39291264, 1.        , 0.24184306,
        0.42361286, 0.31959937, 0.37073365, 0.23956916, 0.12651922,
        0.19373888, 0.21805643, 0.28751279, 0

#### Pearson Similarity

In [10]:
pearson_matrix = np.array(pearson_matrix)
pearson_matrix

array([[ 1.        , -0.17074744,  0.51227193,  0.48419665,  0.17913943,
         0.74364792,  0.31459651,  0.36114435,  0.59044508,  0.24325677,
         0.62085105,  0.37847063,  0.36072354,  0.66788536, -0.00828429,
         0.2318504 , -0.12385396,  0.34756845,  0.35933243,  0.05076569],
       [-0.17074744,  1.        , -0.14175426, -0.56639982,  0.44557791,
         0.01921074,  0.32123214,  0.30148171,  0.3070794 ,  0.36790616,
         0.3564081 ,  0.26449499,  0.10129238,  0.18566926,  0.25445668,
        -0.04584175,  0.327884  ,  0.0818066 , -0.10052003,  0.13387902],
       [ 0.51227193, -0.14175426,  1.        ,  0.18536502,  0.38566704,
         0.39020822,  0.13622063,  0.37127174,  0.54101748,  0.41749117,
         0.57021183,  0.22206787,  0.48190372,  0.41513732,  0.37139068,
         0.44944753, -0.27459604,  0.76117707,  0.62719973,  0.13793103],
       [ 0.48419665, -0.56639982,  0.18536502,  1.        , -0.05887344,
         0.17844523,  0.11039771,  0.18352212, -

In [11]:

rc_vector = df.iloc[:,1:].to_numpy()
rc_vector

array([[ 4.,  5.,  3., nan,  2., nan, nan,  5., nan, nan,  3.,  4., nan,
        nan,  5., nan, nan, nan, nan, nan],
       [ 3.,  4.,  5.,  5.,  5.,  5.,  5.,  4.,  4.,  4.,  5.,  5.,  5.,
         5.,  5.,  5.,  5.,  4.,  4.,  4.],
       [ 5.,  5.,  3., nan,  5., nan,  5.,  3., nan, nan,  5., nan,  1.,
        nan, nan, nan, nan, nan, nan, nan],
       [ 5., nan, nan, nan, nan, nan, nan,  3., nan, nan, nan, nan, nan,
        nan,  3., nan, nan, nan, nan,  3.],
       [nan, nan,  4., nan,  5., nan,  3.,  5., nan, nan,  4., nan,  3.,
         5.,  2.,  4., nan, nan, nan, nan],
       [ 3.,  5.,  4.,  1.,  1.,  5.,  2.,  4.,  3., nan,  4.,  5., nan,
        nan,  4., nan, nan, nan, nan, nan],
       [nan, nan,  4., nan,  3., nan, nan,  3., nan, nan, nan, nan,  3.,
        nan,  3.,  3., nan, nan, nan, nan],
       [nan, nan,  4., nan,  4., nan,  5.,  4., nan, nan, nan, nan, nan,
        nan,  5.,  2., nan, nan, nan, nan],
       [nan,  5.,  5., nan,  5., nan,  5.,  5., nan, nan,  3.,  

##### Raw Cosine Similarity

In [19]:

# this might not be the most optimal approach but I have stacked vector x and vector y as columns first.
# Now that the 0th column respresents first rating and 1st column represents second rating, 
# we have a pairs of ratings in the form of a rows. 
# So if either of those ratings in a pair are Nan, I can exclude the entire row from consideration.
# After removing such pairs, I have split the matrix across columns (2 columns) to get separate vectors x and y.
def rawcosine(x,y):
    colstack = np.column_stack((x,y))
    colstack = np.array(colstack)
    colstack = colstack[~np.isnan(colstack).any(axis=1)]
    x, y = np.array_split(ary = colstack, indices_or_sections = 2, axis=1)

    x = np.squeeze(np.asarray(x))
    y = np.squeeze(np.asarray(y))
    
    sim = np.dot(x,y)/(norm(x)*norm(y)) 
    return sim



In [20]:

rawcosine_matrix = [[0]*rows for k in range(rows)]
for i in range(rows):
    for j in range(rows):
        rawcosine_matrix[i][j] = rawcosine(rc_vector[i], rc_vector[j])

rawcosine_matrix = np.array(rawcosine_matrix)
rawcosine_matrix

/var/folders/tc/04_ll5rn2wd6y60sbby3n1sc0000gp/T/ipykernel_669/1581582207.py:18: RuntimeWarning: invalid value encountered in double_scalars
  sim = np.dot(x,y)/(norm(x)*norm(y))


array([[1.        , 0.93620653, 0.92245569, 0.93856382, 0.87686678,
        0.97252195, 0.92222467, 0.9584769 , 0.93613767, 0.93193853,
        0.91830473, 0.96867751, 0.92067866, 0.93360678, 1.        ,
        0.89451592, 0.96354608, 0.94668481, 0.94523009, 0.924514  ],
       [0.93620653, 1.        , 0.91844479, 0.9217648 , 0.96058996,
        0.91365291, 0.99200384, 0.9672714 , 0.97559774, 0.98817391,
        0.98182067, 0.93261866, 0.97747124, 0.97965387, 1.        ,
        0.96039131, 0.98652088, 0.97171052, 0.97486133, 0.9953452 ],
       [0.92245569, 0.91844479, 1.        , 1.        , 0.92827912,
        0.88758009, 0.89661096, 0.97933935, 0.95430641, 0.98690167,
        0.97439457, 0.92904746, 0.98198051, 0.96566078, 0.9701425 ,
        0.97473459, 0.99469179, 0.96813124, 0.92583809, 0.92737392],
       [0.93856382, 0.9217648 , 1.        , 1.        , 0.91914503,
        0.92883474, 1.        , 0.99388373, 0.9701425 , 1.        ,
        1.        , 0.98058068, 0.9701425 , 0

### Find the most similar n users
Functions to be used to retrieve top-n similar users for one reference user.
(Note that in actual recommender systems we do this for all users at once.)

In [21]:
#Helper function that takes a user id and retrieves the index

def user_to_key(ref_user):
    ref_user_key = df.loc[df['ID'] == ref_user]
    key = ref_user_key.index[0]
    
    return key
    

In [45]:

def sim_users(df, topn_users, similarity, ref_user):
    ref_usr_key = user_to_key(ref_user)
    
    if similarity == 'cosine':
        sim_score_list = cosine_matrix[ref_usr_key]
    elif similarity == 'pearson':
        sim_score_list = pearson_matrix[ref_usr_key]
    elif similarity == 'rawcosine':
        sim_score_list = rawcosine_matrix[ref_usr_key]
     
    sim_score = [(i,score) for (i,score) in enumerate(sim_score_list)]
    del sim_score[ref_usr_key]
    top_sim_values = sorted(sim_score, reverse=True, key = lambda s: s[1])
    top_sim_values = top_sim_values[:topn_users]
    return [(df['ID'].iloc[k], s) for (k, s) in top_sim_values]


### Find predicted ratings for our reference user

Functions that return the rating predictions for the missing item ratings of a user (ref_user) given their top-n neighbors (topn_usrs). 

In [47]:

def avg_rating(u_vector, simU, item, avgtype):
    if avgtype == "simple":
        return np.array([u_vector[user_to_key(u)][item] for (u,_) in simU]).mean() #simple average
    elif avgtype == "weighted":
        ratingvec = [u_vector[user_to_key(u)][item] for (u,_) in simU]  
        simvec = [x[1] for x in simU]
        return np.dot(ratingvec,simvec)/sum(simvec) #weighted average

In [48]:
#Function that takes as input a reference user (ref_user) and their top-n neighborhood (topn_usrs) and returns an 
#array containing the predicted ratings for the items not yet rated by the user

def pred_ratings(u_vector, ref_user, topn_usrs, avg_type):   
    ref_usr_key = user_to_key(ref_user)
    
    pred = np.zeros(len(u_vector[ref_usr_key]))
    
    for i in range(len(u_vector[ref_usr_key])):
        if(u_vector[ref_usr_key][i] == 0):   #calculate predictions only for non-rated items
            pred[i] = avg_rating(u_vector, topn_usrs, i, avg_type)  #TODO: update as needed, also changed in def pred_ratings
        else:
            pred[i] = None   #Previously rated items will turn to "nan". Note that this is one way of many to deal with this. 
            
    return pred

    

#### Predict ratings for reference user

Toy example with one user as input. Other hyperparameters can be set here. 

In [49]:
topn = 5
similarity = 'cosine'
avg_type = "weighted"

top_neighbors = sim_users(df, topn, similarity, 13)    
    
pred = pred_ratings(u_vector, 13, top_neighbors, avg_type)

pred

array([2.01657289,        nan,        nan, 1.75377575,        nan,
              nan,        nan,        nan, 1.53603311, 0.96143503,
       4.2288732 ,        nan,        nan, 1.92269233, 3.1315763 ,
       2.95301418,        nan,        nan,        nan,        nan])

### Get Recommendations

Functions that help demonstrate the final step of the recommender system. Once the predicted ratings are available, the system retrieves the names of the respective movies (items) and presents them to the user as recommendations. 

#### Movie name cleaning

In [50]:
def getMovieName(movie_id):
    if movie_id not in movie_dict:
        return ""
    m = movie_dict[movie_id].split('[')
    temp = m[1].split(']')
    return temp[0]

#### Getting Recommendations

In [51]:
def getMovieRecommendationsForUser(userId, recommendations):
    if userId not in user_dict:
        print("User id is not present")
        return
    u_id = user_dict[userId]
    recommended_movies = recommendations[u_id]
    movie_list = []
    for movie in recommended_movies:
        movie_list.append((getMovieName(movie[0]),movie[1]))
    return movie_list

#### Creating Dictionary for top movie recommendations

In [52]:
from collections import defaultdict
import math

def getMovieRecommendations(uid, topN):
    top_recs = defaultdict(list)
    i=1
    for pred in predictions:
        if(math.isnan(pred)==False):
            top_recs[uid].append((i, pred))
        i=i+1
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs 

In [53]:
predictions = list(pred)

In [54]:
recommendations = getMovieRecommendations(13, 3)

In [55]:
getMovieRecommendationsForUser(13,recommendations)

[('The hunger games', 4.2288731974417795),
 ('Twilight', 3.1315763015955507),
 ('Lala Land', 2.9530141792379556)]

In [64]:

## A. Pearson, top-5 neighbors, weighted average
## B. Raw cosine, top-5 neighbors, simple average
## C. Raw cosine, top-5 neighbors, weighted average
##
## Do you notice big deviations in the results (movies and/or ratings)?
##

topn = 5
similarity = 'pearson'
avg_type = "weighted"
top_neighbors = sim_users(df, topn, similarity, 13)       
pred = pred_ratings(u_vector, 13, top_neighbors, avg_type)
predictions = list(pred)
recommendations = getMovieRecommendations(13, 3)
getMovieRecommendationsForUser(13,recommendations)

[('Lala Land', 2.329404940803179),
 ('Twilight', 2.0791142566413328),
 ('The hunger games', 1.7036824183717445)]

In [57]:
##TODO: B. Raw cosine, top-5 neighbors, simple average
topn = 5
similarity = 'rawcosine'
avg_type = "simple"
top_neighbors = sim_users(df, topn, similarity, 13)       
pred = pred_ratings(u_vector, 13, top_neighbors, avg_type)
predictions = list(pred)
recommendations = getMovieRecommendations(13, 3)
getMovieRecommendationsForUser(13,recommendations)

[('Lala Land', 3.6), ('500 days of Summer', 3.0), ('The hunger games', 2.8)]

In [61]:
##TODO: C. Raw cosine, top-5 neighbors, weighted average
topn = 5
similarity = 'rawcosine'
avg_type = "weighted"
top_neighbors = sim_users(df, topn, similarity, 13)       
pred = pred_ratings(u_vector, 13, top_neighbors, avg_type)
predictions = list(pred)
recommendations = getMovieRecommendations(13, 3)
getMovieRecommendationsForUser(13,recommendations)

[('Lala Land', 3.5966158238542714),
 ('500 days of Summer', 2.998087363467243),
 ('Twilight', 2.7997822891060844)]

In [62]:
#TODO: Raw cosine, top-5 neighbors, weighted average, 4 sets
topn = 5
similarity = 'rawcosine'
avg_type = "weighted"
top_neighbors = sim_users(df, topn, similarity, 13)       
pred = pred_ratings(u_vector, 13, top_neighbors, avg_type)
predictions = list(pred)
recommendations = getMovieRecommendations(13, 4)
getMovieRecommendationsForUser(13,recommendations)

[('Lala Land', 3.5966158238542714),
 ('500 days of Summer', 2.998087363467243),
 ('Twilight', 2.7997822891060844),
 ('The hunger games', 2.7981655554271265)]